In [26]:
import pickle
import pandas
import tensorflow as tf 
from tensorflow.keras.models import load_model

In [27]:
# Loading the model
classifi_model = load_model('bank_churn_classification.h5')

In [28]:
# Loading the data pre processing pickel files

with open('label_encoder_gender.pkl', 'rb') as file:
    le_gender = pickle.load(file)


with open('ohe_encoder_geography.pkl', 'rb') as file:
    ohe_geography = pickle.load(file)


with open('standard_scaler.pkl', 'rb') as file:
    ss_all = pickle.load(file)


In [29]:
# Testing data point - JSON

test_ip_data = {
    'CreditScore': 870,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 46,
    'Tenure': 3,
    'Balance': 6009874,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 5500000
}

In [30]:
# Applying data tranformation

geo_encode = ohe_geography.transform([[test_ip_data['Geography']]]).toarray()
geo_encode_df = pandas.DataFrame(geo_encode, columns=ohe_geography.get_feature_names_out(['Geography']))
geo_encode_df

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [31]:
test_ip_data = pandas.DataFrame([test_ip_data])

In [32]:
df_test_ip_data = pandas.concat([test_ip_data.reset_index(drop=True),geo_encode_df], axis=1)
df_test_ip_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,870,France,Female,46,3,6009874,2,1,1,5500000,1.0,0.0,0.0


In [33]:
df_test_ip_data['Gender'] = le_gender.transform(df_test_ip_data['Gender'])
df_test_ip_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,870,France,0,46,3,6009874,2,1,1,5500000,1.0,0.0,0.0


In [34]:
df_test_ip_data.drop('Geography', axis=1, inplace=True)
df_test_ip_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,870,0,46,3,6009874,2,1,1,5500000,1.0,0.0,0.0


In [35]:
# Applying scaling
df_transformed = ss_all.transform(df_test_ip_data)
df_transformed

array([[ 2.26943823, -1.0862028 ,  0.66329355, -0.70388869, 95.34312591,
         0.81118618,  0.64920267,  0.96510833, 93.80030079,  1.0010005 ,
        -0.58408625, -0.57138416]])

In [37]:
# Prediction
prediction = classifi_model.predict(df_transformed)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


array([[1.6655006e-12]], dtype=float32)

In [39]:
pred_prob = prediction[0][0]
pred_prob

np.float32(1.6655006e-12)

In [40]:
if pred_prob > 0.5:
    print("Customer is likely to churn")
else:
    print("Customer is not likely to churn")

Customer is not likely to churn
